## Imports 

In [1]:
import os
import datetime
import pikepdf
import pandas as pd
import numpy as np
import duckdb
from pandasql import sqldf
from pathlib import Path

## Leitura

In [2]:
# Ler o arquivo Excel para um DataFrame Pandas
raw = pd.read_excel("EventosExcecionaisNew.xlsx")
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão
0,EDPD_2018_JAN_T_1,Almada,02/01/2018,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado
1,EDPD_2018_JAN_T_2,Serta,02/01/2018,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
2,EDPD_2018_JAN_T_3,Felgueiras,04/01/2018,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
3,EDPD_2018_JAN_T_4,Torres Vedras,05/01/2018,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado
4,EDPD_2018_JAN_T_5,Nisa,09/01/2018,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,29/11/2017,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,03/11/2017,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,23/11/2017,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado
7563,EDPD_2017_NOV_T_65,Almeirim,03/11/2017,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado


In [3]:
raw.iloc[3359]

Código do Relatório                                             EDPD_2014_JUN_T_020
Concelho origem do incidente                               Freixo De Espada A Cinta
Data do incidente                                                        2014-05-12
Nível de Tensão                                                                  BT
Causa do incidente                             Trabalhos De Terceiros ( Gruas,... )
Duração incidente (min)                                                          98
N.º clientes afetados                                                           6.0
TIEPI MT (min)                                                                    0
SAIFI MT (#)                                                                      0
SAIDI MT (min)                                                                    0
END (MWh)                                                                         0
SAIFI BT (#)                                                               0

In [4]:
raw.dtypes

Código do Relatório                object
Concelho origem do incidente       object
Data do incidente                  object
Nível de Tensão                    object
Causa do incidente                 object
Duração incidente (min)            object
N.º clientes afetados             float64
TIEPI MT (min)                     object
SAIFI MT (#)                       object
SAIDI MT (min)                     object
END (MWh)                          object
SAIFI BT (#)                       object
SAIDI BT (min)                     object
Qualidade de Energia Elétrica      object
Qualidade de Serviço Comercial     object
Fundamentação                      object
Decisão                            object
dtype: object

## Colunas Inuteis

In [5]:
raw["Qualidade de Energia Elétrica"].fillna("n.a.", inplace=True)
raw["Qualidade de Serviço Comercial"].fillna("n.a.", inplace=True)

In [6]:
# Função para converter o código do relatório em mês
def codigo_para_mes(codigo):
    # Dividir o código pelo caractere "_"
    partes = codigo.split("_")
    # Obter o terceiro elemento (índice 2) e converter para mês
    mes_str = partes[2][:3].upper()  # Pegar apenas as 3 primeiras letras do mês e converter para maiúsculas
    meses = {
        "JAN": 1, "FEV": 2, "MAR": 3, "ABR": 4, "MAI": 5, "JUN": 6,
        "JUL": 7, "AGO": 8, "SET": 9, "OUT": 10, "NOV": 11, "DEZ": 12
    }
    # Retornar o número do mês correspondente
    return meses.get(mes_str, None)

# Aplicar a função à coluna "Código do Relatório" e criar uma nova coluna "Mês"
raw['MêsCod'] = raw['Código do Relatório'].apply(codigo_para_mes)
raw['MêsCod'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

## Coluna *Data do Incidente* ---

In [7]:
def format_date(dateint, mes_cod):

    date_str = str(dateint)
    parts = date_str.split("-")
    if len(parts) != 3:
        return dateint  # Mantém a data inalterada se não tiver 3 partes separadas por "-"

    try:
        dataCorreta = datetime.datetime(int(parts[0]), int(parts[1]), int(parts[2]))
        return dataCorreta.strftime("%d/%m/%Y")
    except ValueError: 
        valores = [int(parts[0]), int(parts[1]), int(parts[2])]
        ano = max(valores)
        valores.remove(ano)
        mes = mes_cod
        valores.remove(mes_cod)
        dataCorreta = datetime.datetime(int(ano), int(mes), int(valores[0]))
        return dataCorreta.strftime("%d/%m/%Y")

raw["Data do incidente"] = raw.apply(lambda row: format_date(row["Data do incidente"], row["MêsCod"]), axis=1)
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod
0,EDPD_2018_JAN_T_1,Almada,02/01/2018,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
1,EDPD_2018_JAN_T_2,Serta,02/01/2018,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
2,EDPD_2018_JAN_T_3,Felgueiras,04/01/2018,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
3,EDPD_2018_JAN_T_4,Torres Vedras,05/01/2018,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
4,EDPD_2018_JAN_T_5,Nisa,09/01/2018,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,29/11/2017,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,03/11/2017,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,23/11/2017,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7563,EDPD_2017_NOV_T_65,Almeirim,03/11/2017,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11


In [8]:
def remover_horas_se_presentes(data_series, formato='%d/%m/%Y %H:%M'):
    try:
        # Converter para datetime com errors='coerce' para lidar com valores inválidos
        converted_dates = pd.to_datetime(data_series, format=formato)
        
        # Remover as horas
        return converted_dates.date()
    except ValueError:
        return data_series

# Aplicar a função à coluna "Data do incidente" da base de dados raw
raw["Data do incidente"] = raw["Data do incidente"].apply(remover_horas_se_presentes)


In [9]:
def excel_to_date(numerox):
    numero = str(numerox)
    # Verificar se o valor pode ser convertido para um número inteiro e se o comprimento é <= 5
    if numero.isdigit() and len(numero) <= 5:
        try:
            # Calcular a data base do Excel
            data_base_excel = datetime.datetime(1899, 12, 30)
            # Adicionar o número de dias à data base do Excel
            data = data_base_excel + datetime.timedelta(days=int(numero))
            return data.strftime("%d/%m/%Y")  # Formatando a data conforme necessário
        except ValueError:
            return numero  # Se ocorrer um erro, retornar o valor original
    else:
        return numerox  # Se não for um número de data do Excel, retornar o valor original

raw["Data do incidente"] = raw["Data do incidente"].apply(excel_to_date)

In [10]:
def converter_data(numeroX):
    numero = str(numeroX)
    if numero.isdigit() and len(numero) > 6:
        try:
            data_formatada = datetime.datetime.strptime(numero, "%Y%m%d").strftime("%d/%m/%Y")
            return data_formatada
        except ValueError:
            return numero
    else:
        return numeroX

raw["Data do incidente"] = raw["Data do incidente"].apply(converter_data)

In [11]:
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod
0,EDPD_2018_JAN_T_1,Almada,02/01/2018,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
1,EDPD_2018_JAN_T_2,Serta,02/01/2018,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
2,EDPD_2018_JAN_T_3,Felgueiras,04/01/2018,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
3,EDPD_2018_JAN_T_4,Torres Vedras,05/01/2018,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
4,EDPD_2018_JAN_T_5,Nisa,09/01/2018,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,29/11/2017,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,03/11/2017,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,23/11/2017,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7563,EDPD_2017_NOV_T_65,Almeirim,03/11/2017,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11


In [12]:
raw.at[1000, 'Data do incidente'] = '29/10/2021'
raw.drop(5325, inplace=True)  # TODO Decidir o que fazer
raw["Data do incidente"] = pd.to_datetime(raw["Data do incidente"], format='%d/%m/%Y')
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
1,EDPD_2018_JAN_T_2,Serta,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11
7563,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11


In [13]:
raw['Dia'] = raw['Data do incidente'].dt.day
raw['Mês'] = raw['Data do incidente'].dt.month
raw['Ano'] = raw['Data do incidente'].dt.year
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod,Dia,Mês,Ano
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1,2,1,2018
1,EDPD_2018_JAN_T_2,Serta,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1,2,1,2018
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1,4,1,2018
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,1,5,1,2018
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,1,9,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11,29,11,2017
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11,3,11,2017
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11,23,11,2017
7563,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,11,3,11,2017


### Datas Incoerentes

In [14]:
# Converter a coluna "Data do incidente" para datetime com errors='coerce'
teste = raw.copy()
teste["Data do incidente"] = pd.to_datetime(teste["Data do incidente"], format='%d/%m/%Y', errors='coerce')

# Verificar as linhas com valores problemáticos (NaT)
linhas_problema = teste[teste["Data do incidente"].isna()]
raw.loc[linhas_problema.index]

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod,Dia,Mês,Ano


In [15]:
linhas_diferentes = raw[raw['Mês'] != raw['MêsCod']]
linhas_diferentes

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,MêsCod,Dia,Mês,Ano
1576,EDPD_2019_FEV_T_7,Vila Pouca De Aguiar,2019-01-07,MT,Abate De Árvores,289,2222.0,"0,0084976","0,0002643","0,0320010",...,"0,0001462","0,0175673",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,7,1,2019
1605,EDPD_2019_FEV_T_36,Pombal,2019-01-05,MT,Abate De Árvores,157,3882.0,"0,0813289","0,0004735","0,0490400",...,"0,0003334","0,0237264",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,5,1,2019
1606,EDPD_2019_FEV_T_37,Braga,2019-01-01,BT,Intervenções À Superficie,157,62.0,"0,0000000","0,0000000","0,0000000",...,"0,0000101","0,0015902",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,2,1,1,2019
1607,EDPD_2019_FEV_T_38,Portimao,2019-01-04,MT,Aves,271,695.0,"0,0122643","0,0001321","0,0288323",...,"0,0000441","0,0085279",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,4,1,2019
1608,EDPD_2019_FEV_T_39,Almeirim,2019-01-03,MT,Aves,119,101.0,"0,0057074","0,0001101","0,0059458",...,"0,0000119","0,0006810",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,3,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,EDPD_2014_JUN_T_014,Oliveira Do Bairro,2014-05-08,BT,Veículos,96,1.0,0,0,0,...,0,"0,000016",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,6,8,5,2014
3359,EDPD_2014_JUN_T_020,Freixo De Espada A Cinta,2014-05-12,BT,"Trabalhos De Terceiros ( Gruas,... )",98,6.0,0,0,0,...,"0,000001","0,000097",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,6,12,5,2014
3371,EDPD_2014_JUN_T_032,Macedo De Cavaleiros,2014-05-02,BT,Veículos,91,12.0,0,0,0,...,"0,000002","0,000179",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,6,2,5,2014
3372,EDPD_2014_JUN_T_033,Mirandela,2014-05-28,BT,Abate De Árvores,123,63.0,0,0,0,...,"0,000010","0,001250",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,6,28,5,2014


In [16]:
for index, row in raw.iterrows():
    # Obter o mês correto de 'MêsCod'
    mes_correto = row['MêsCod']
    
    # Atualizar o mês na coluna 'Data do incidente' com o valor correto
    raw.at[index, 'Data do incidente'] = row['Data do incidente'].replace(month=mes_correto)
raw['Dia'] = raw['Data do incidente'].dt.day
raw['Mês'] = raw['Data do incidente'].dt.month
raw['Ano'] = raw['Data do incidente'].dt.year
raw.drop("MêsCod", axis=1, inplace=True)
raw.iloc[3373]

Código do Relatório                                             EDPD_2014_JUN_T_034
Concelho origem do incidente                                      Torre De Moncorvo
Data do incidente                                               2014-06-23 00:00:00
Nível de Tensão                                                                  BT
Causa do incidente                                           Malfeitoria/Vandalismo
Duração incidente (min)                                                          88
N.º clientes afetados                                                           3.0
TIEPI MT (min)                                                                    0
SAIFI MT (#)                                                                      0
SAIDI MT (min)                                                                    0
END (MWh)                                                                         0
SAIFI BT (#)                                                                

## Coluna *Código do Relatório* ---

In [17]:
duplicados = raw[raw.duplicated(subset=['Código do Relatório'], keep=False)]
duplicados

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano
3048,EDPD_2014_ABR_T_ 001,Cantanhede,2014-04-02,BT,Veículos,186,32.0,0,0,0,0,"0,000005","0,000986",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,2,4,2014
3049,EDPD_2014_ABR_T_ 002,Tondela,2014-04-03,BT,Veículos,115,20.0,0,0,0,0,"0,000003","0,000381",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,3,4,2014
3050,EDPD_2014_ABR_T_ 003,Caldas Da Rainha,2014-04-05,BT,Malfeitoria/Vandalismo,195,2.0,0,0,0,0,"0,000004","0,000871",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,5,4,2014
3051,EDPD_2014_ABR_T_ 004,Viseu,2014-04-05,BT,Veículos,86,5.0,0,0,0,0,"0,000007","0,000625",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,5,4,2014
3052,EDPD_2014_ABR_T_ 005,Nazare,2014-04-07,BT,Escavações,160,3.0,0,0,0,0,"0,000005","0,000872",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,7,4,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5868,EDPD_2016_MAI_T_109,Sesimbra,2016-05-26,BT,Malfeitoria/Vandalismo,58,4.0,0,0,0,0,"0,00000","0,00004",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,26,5,2016
5869,EDPD_2016_MAI_T_109,Sesimbra,2016-05-26,BT,Malfeitoria/Vandalismo,65,3.0,0,0,0,0,"0,00000","0,00003",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,26,5,2016
5870,EDPD_2016_MAI_T_109,Sesimbra,2016-05-28,BT,Malfeitoria/Vandalismo,88,3.0,0,0,0,0,"0,00000","0,00004",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,28,5,2016
5871,EDPD_2016_MAI_T_109,Sesimbra,2016-05-28,BT,Malfeitoria/Vandalismo,102,3.0,0,0,0,0,"0,00000","0,00005",n.a.,n.a.,Não reúne condições para aprovação: -Não cumpr...,Não Aprovado,28,5,2016


In [18]:
raw.drop_duplicates(subset=['Código do Relatório'], inplace=True)
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018
1,EDPD_2018_JAN_T_2,Serta,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017
7561,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017
7562,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017
7563,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017


## Coluna *Concelho origem do incidente* ---

In [19]:
correcoes = {
    "Almada": "Almada",
    "Serta": "Sertã",
    "Felgueiras": "Felgueiras",
    "Torres Vedras": "Torres Vedras",
    "Nisa": "Nisa",
    "Esposende": "Esposende",
    "Idanha-A-Nova": "Idanha-a-Nova",
    "Arraiolos": "Arraiolos",
    "Amarante": "Amarante",
    "Setubal": "Setúbal",
    "Lamego": "Lamego",
    "Agueda": "Águeda",
    "Ovar": "Ovar",
    "Espinho": "Espinho",
    "Lagos": "Lagos",
    "Vila Vicosa": "Vila Viçosa",
    "Sabugal": "Sabugal",
    "Celorico Da Beira": "Celorico da Beira",
    "Meda": "Mêda",
    "Viseu": "Viseu",
    "Rio Maior": "Rio Maior",
    "Marco De Canaveses": "Marco de Canaveses",
    "Vouzela": "Vouzela",
    "Montemor-O-Novo": "Montemor-o-Novo",
    "Vila Nova De Cerveira": "Vila Nova de Cerveira",
    "Gondomar": "Gondomar",
    "Mirandela": "Mirandela",
    "Satao": "Sátão",
    "Tondela": "Tondela",
    "Vieira Do Minho": "Vieira do Minho",
    "Montalegre": "Montalegre",
    "Vila Real": "Vila Real",
    "Oliveira Do Bairro": "Oliveira do Bairro",
    "Vila Nova De Famalicao": "Vila Nova de Famalicão",
    "Estarreja": "Estarreja",
    "Leiria": "Leiria",
    "Amadora": "Amadora",
    "Lisboa": "Lisboa",
    "Penacova": "Penacova",
    "Barcelos": "Barcelos",
    "Redondo": "Redondo",
    "Avis": "Avis",
    "Cadaval": "Cadaval",
    "Ponte Da Barca": "Ponte da Barca",
    "Ponte De Lima": "Ponte de Lima",
    "Alcobaca": "Alcobaça",
    "Figueira Da Foz": "Figueira da Foz",
    "Soure": "Soure",
    "Viana Do Castelo": "Viana do Castelo",
    "Pombal": "Pombal",
    "Lousa": "Lousã",
    "Coruche": "Coruche",
    "Cinfaes": "Cinfães",
    "Maia": "Maia",
    "Povoa De Varzim": "Póvoa de Varzim",
    "Cantanhede": "Cantanhede",
    "Faro": "Faro",
    "Mortagua": "Mortágua",
    "Arcos De Valdevez": "Arcos de Valdevez",
    "Abrantes": "Abrantes",
    "Porto": "Porto",
    "Aljezur": "Aljezur",
    "Mafra": "Mafra",
    "Torres Novas": "Torres Novas",
    "Povoa De Lanhoso": "Póvoa de Lanhoso",
    "Chaves": "Chaves",
    "Arronches": "Arronches",
    "Vila Nova Da Barquinha": "Vila Nova da Barquinha",
    "Ponte De Sor": "Ponte de Sôr",
    "Cartaxo": "Cartaxo",
    "Tomar": "Tomar",
    "Silves": "Silves",
    "Aljustrel": "Aljustrel",
    "Pampilhosa Da Serra": "Pampilhosa da Serra",
    "Coimbra": "Coimbra",
    "Fundao": "Fundão",
    "Alcacer Do Sal": "Alcácer do Sal",
    "Montemor-O-Velho": "Montemor-o-Velho",
    "Alter Do Chao": "Alter do Chão",
    "Seixal": "Seixal",
    "Nelas": "Nelas",
    "Mealhada": "Mealhada",
    "Ourem": "Ourém",
    "Castro Daire": "Castro Daire",
    "Vila Pouca De Aguiar": "Vila Pouca de Aguiar",
    "Ilhavo": "Ílhavo",
    "Sabrosa": "Sabrosa",
    "Barreiro": "Barreiro",
    "Moncao": "Monção",
    "Salvaterra De Magos": "Salvaterra de Magos",
    "Benavente": "Benavente",
    "Beja": "Beja",
    "Valenca": "Valença",
    "Seia": "Seia",
    "Santo Tirso": "Santo Tirso",
    "Castelo Branco": "Castelo Branco",
    "Condeixa-A-Nova": "Condeixa-a-Nova",
    "Castro Verde": "Castro Verde",
    "Loures": "Loures",
    "Tabua": "Tábua",
    "Vila Nova De Gaia": "Vila Nova de Gaia",
    "Tavira": "Tavira",
    "Olhao": "Olhão",
    "Mertola": "Mértola",
    "Alenquer": "Alenquer",
    "Baiao": "Baião",
    "Sintra": "Sintra",
    "Evora": "Évora",
    "Portalegre": "Portalegre",
    "Sardoal": "Sardoal",
    "Amares": "Amares",
    "Albufeira": "Albufeira",
    "Almodovar": "Almodôvar",
    "Fafe": "Fafe",
    "Guimaraes": "Guimarães",
    "Penafiel": "Penafiel",
    "Gaviao": "Gavião",
    "Matosinhos": "Matosinhos",
    "Alijo": "Alijó",
    "Loule": "Loulé",
    "Portimao": "Portimão",
    "Moura": "Moura",
    "Ferreira Do Alentejo": "Ferreira do Alentejo",
    "Odemira": "Odemira",
    "Estremoz": "Estremoz",
    "Almeirim": "Almeirim",
    "Trofa": "Trofa",
    "Caldas Da Rainha": "Caldas da Rainha",
    "Valpacos": "Valpaços",
    "Castanheira De Pera": "Castanheira de Pêra",
    "Mangualde": "Mangualde",
    "Aveiro": "Aveiro",
    "Braga": "Braga",
    "Sobral De Monte Agraco": "Sobral de Monte Agraço",
    "Santa Maria Da Feira": "Santa Maria da Feira",
    "Vendas Novas": "Vendas Novas",
    "Alvaiazere": "Alvaiázere",
    "Serpa": "Serpa",
    "Sines": "Sines",
    "Entroncamento": "Entroncamento",
    "Alcanena": "Alcanena",
    "Azambuja": "Azambuja",
    "Sao Joao Da Pesqueira": "São João da Pesqueira",
    "Miranda Do Corvo": "Miranda do Corvo",
    "Penamacor": "Penamacor",
    "Mogadouro": "Mogadouro",
    "Trancoso": "Trancoso",
    "Peniche": "Peniche",
    "Paredes": "Paredes",
    "Vidigueira": "Vidigueira",
    "Barrancos": "Barrancos",
    "Macedo De Cavaleiros": "Macedo de Cavaleiros",
    "Proenca-A-Nova": "Proença-a-Nova",
    "Ourique": "Ourique",
    "Alcochete": "Alcochete",
    "Monchique": "Monchique",
    "Vila De Rei": "Vila de Rei",
    "Guarda": "Guarda",
    "Lagoa": "Lagoa",
    "Vale De Cambra": "Vale de Cambra",
    "Montijo": "Montijo",
    "Macao": "Mação",
    "Murtosa": "Murtosa",
    "Ribeira De Pena": "Ribeira de Pena",
    "Braganca": "Bragança",
    "Oeiras": "Oeiras",
    "Pedrogao Grande": "Pedrógão Grande",
    "Viana Do Alentejo": "Viana do Alentejo",
    "Arouca": "Arouca",
    "Valongo": "Valongo",
    "Palmela": "Palmela",
    "Moimenta Da Beira": "Moimenta da Beira",
    "Boticas": "Boticas",
    "Alvito": "Alvito",
    "Ansiao": "Ansião",
    "Torre De Moncorvo": "Torre de Moncorvo",
    "Castro Marim": "Castro Marim",
    "Sesimbra": "Sesimbra",
    "Cascais": "Cascais",
    "Mira": "Mira",
    "Porto De Mos": "Porto de Mós",
    "Vila Franca De Xira": "Vila Franca de Xira",
    "Moita": "Moita",
    "Marinha Grande": "Marinha Grande",
    "Vila Do Bispo": "Vila do Bispo",
    "Carregal Do Sal": "Carregal do Sal",
    "Vila Do Conde": "Vila do Conde",
    "Borba": "Borba",
    "Elvas": "Elvas",
    "Vila Verde": "Vila Verde",
    "Odivelas": "Odivelas",
    "Oliveira Do Hospital": "Oliveira do Hospital",
    "Sever Do Vouga": "Sever do Vouga",
    "Território De Portugal Continental": "Território de Portugal Continental",
    "Covilha": "Covilhã",
    "Ferreira Do Zezere": "Ferreira do Zêzere",
    "Penalva Do Castelo": "Penalva do Castelo",
    "Peso Da Regua": "Peso da Régua",
    "Santiago Do Cacem": "Santiago do Cacém",
    "Pinhel": "Pinhel",
    "Gois": "Góis",
    "Portel": "Portel",
    "Fornos De Algodres": "Fornos de Algodres",
    "Santarem": "Santarém",
    "Lousada": "Lousada",
    "Golega": "Golegã",
    "Albergaria-A-Velha": "Albergaria-a-Velha",
    "Oleiros": "Oleiros",
    "Sao Bras De Alportel": "São Brás de Alportel",
    "Cabeceiras De Basto": "Cabeceiras de Basto",
    "Almeida": "Almeida",
    "Anadia": "Anadia",
    "Carrazeda De Ansiaes": "Carrazeda de Ansiães",
    "Vila Nova De Foz Coa": "Vila Nova de Foz Côa",
    "Belmonte": "Belmonte",
    "Portugal Continental": "Portugal Continental",
    "Mourao": "Mourão",
    "Sousel": "Sousel",
    "Nazare": "Nazaré",
    "Celorico De Basto": "Celorico de Basto",
    "Arganil": "Arganil",
    "Alandroal": "Alandroal",
    "Murca": "Murça",
    "Território De Portugal Co": "Território de Portugal Continental",
    "Alpiarca": "Alpiarça",
    "Monforte": "Monforte",
    "Chamusca": "Chamusca",
    "Covilhã": "Covilhã",
    "Reguengos De Monsaraz": "Reguengos de Monsaraz",
    "São João Da Pesqueira": "São João da Pesqueira",
    "Portimão": "Portimão",
    "Alcácer Do Sal": "Alcácer do Sal",
    "Lagoa (Faro)": "Lagoa",
    "Porto De Mós": "Porto de Mós",
    "Santarém": "Santarém",
    "Évora": "Évora",
    "Setúbal": "Setúbal",
    "Bragança": "Bragança",
    "Mértola": "Mértola",
    "Águeda": "Águeda",
    "Aguiar Da Beira": "Aguiar da Beira",
    "Ílhavo": "Ílhavo",
    "Idanha‐A‐Nova": "Idanha-a-Nova",
    "Montemor‐O‐Novo": "Montemor-o-Novo",
    "Alter Do Chão": "Alter do Chão",
    "Almodôvar": "Almodôvar",
    "Oliveira De Azeméis": "Oliveira de Azeméis",
    "Sátão": "Sátão",
    "Alcobaça": "Alcobaça",
    "Albergaria‐A‐Velha": "Albergaria-a-Velha",
    "Santiago Do Cacém": "Santiago do Cacém",
    "Póvoa De Varzim": "Póvoa de Varzim",
    "Gouveia": "Gouveia",
    "Sertã": "Sertã",
    "Vila Nova De Paiva": "Vila Nova de Paiva",
    "Ourém": "Ourém",
    "São João Da Madeira": "São João da Madeira",
    "Póvoa De Lanhoso": "Póvoa de Lanhoso",
    "Olhão": "Olhão",
    "Loulé": "Loulé",
    "Lousã": "Lousã",
    "Fundão": "Fundão",
    "Vagos": "Vagos",
    "Condeixa‐A‐Nova": "Condeixa-a-Nova",
    "Bombarral": "Bombarral",
    "Vila Nova De Famalicão": "Vila Nova de Famalicão",
    "Fronteira": "Fronteira",
    "Mortágua": "Mortágua",
    "Sobral De Monte Agraço": "Sobral de Monte Agraço",
    "São Pedro Do Sul": "São Pedro do Sul",
    "Guimarães": "Guimarães",
    "Montemor‐O‐Velho": "Montemor-o-Velho",
    "Figueiro Dos Vinhos": "Figueiró dos Vinhos",
    "Crato": "Crato",
    "Vila Real De Santo Antonio": "Vila Real de Santo António",
    "Oliveira De Azemeis": "Oliveira de Azeméis",
    "Vinhais": "Vinhais",
    "Tabuaco": "Tabuaço",
    "Batalha": "Batalha",
    "Caminha": "Caminha",
    "Santa Comba Dao": "Santa Comba Dão",
    "Vila Velha De Rodao": "Vila Velha de Ródão",
    "Melgaco": "Melgaço",
    "Grandola": "Grândola",
    "Pacos De Ferreira": "Paços de Ferreira",
    "Região Centro De P": "Região Centro de Portugal",
    "Alijó": "Alijó",
    "Grândola": "Grândola",
    "Tábua": "Tábua",
    "Figueiró Dos Vinhos": "Figueiró dos Vinhos",
    "Valença": "Valença",
    "Proença‐A‐Nova": "Proença-a-Nova",
    "Terras De Bouro": "Terras de Bouro",
    "Vizela": "Vizela",
    "Cuba": "Cuba",
    "Tabuaço": "Tabuaço",
    "Paços De Ferreira": "Paços de Ferreira",
    "Miranda Do Douro": "Miranda do Douro",
    "Ferreira Do Zêzere": "Ferreira do Zêzere",
    "Golegã": "Golegã",
    "Figueira De Castelo Rodrigo": "Figueira de Castelo Rodrigo",
    "Góis": "Góis",
    "Santa Comba Dão": "Santa Comba Dão",
    "Mação": "Mação",
    "Sernancelhe": "Sernancelhe",
    "Resende": "Resende",
    "Vila Viçosa": "Vila Viçosa",
    "Ansião": "Ansião",
    "Monção": "Monção",
    "(Distritos) Aveiro, Castelo Branco, Portalegre, Porto, Santarém E Viseu": "(Distritos) Aveiro, Castelo Branco, Portalegre, Porto, Santarém e Viseu",
    "Penela": "Penela",
    "Mora": "Mora",
    "Armamar": "Armamar",
    "Casrelo Branco": "Castelo Branco",
    "Vila Nova De Poiares": "Vila Nova de Poiares",
    "Sao Pedro Do Sul": "São Pedro do Sul",
    "Vila Nova De Ourem": "Ourém",
    "Tarouca": "Tarouca",
    "Paredes De Coura": "Paredes de Coura",
    "Alfandega Da Fe": "Alfândega da Fé",
    "Obidos": "Óbidos",
    "Lourinha": "Lourinhã",
    "Sao Joao Da Madeira": "São João da Madeira",
    "Oliveira De Frades": "Oliveira de Frades",
    "Santa Marta De Penaguiao": "Santa Marta de Penaguião",
    "Peso Da Régua": "Peso da Régua",
    "Mondim De Basto": "Mondim de Basto",
    "Oliveirinha Do Bairro": "Oliveira do Bairro",
    "Mesao Frio": "Mesão Frio",
    "Freixo De Espada A Cinta": "Freixo de Espada à Cinta",
    "Proença-A-Nova": "Proença-a-Nova",
    "Arruda Dos Vinhos": "Arruda dos Vinhos",
    "Castelo De Vide": "Castelo de Vide",
    "Marvao": "Marvão",
    "Constancia": "Constância",
    "Campo Maior": "Campo Maior",
    "Distritos De Beja E Faro": "Distritos de Beja e Faro",
    "Murça": "Murça",
    "Distritos De Aveiro, Braga, Bragança, Castelo Branco, Coimbra, Guarda,  Porto, Vila Real E Viseu": "Distritos de Aveiro, Braga, Bragança, Castelo Branco, Coimbra, Guarda, Porto, Vila Real e Viseu",
    "Castelo De Paiva": "Castelo de Paiva",
    "Distritos De Aveiro, Braga, Castelo Branco, Coimbra, Évora, Faro, Guarda, Leiria, Portalegre, Porto, Viana Do Castelo, Vila Real E Viseu": "Distritos de Aveiro, Braga, Castelo Branco, Coimbra, Évora, Faro, Guarda, Leiria, Portalegre, Porto, Viana do Castelo, Vila Real e Viseu",
    "Vila Flor": "Vila Flor",
    "Vimioso": "Vimioso",
    "Tavira/ Silves/ Loulé/ Faro/ Serpa/ Lagoa/ Beja": "Tavira, Silves, Loulé, Faro, Serpa, Lagoa, Beja",
    "Penedono": "Penedono",
    "Óbidos": "Óbidos",
    "Região Norte": "Região Norte",
    "Região Aveiro - Feira": "Região Aveiro - Feira",
    "Região Leiria - Caldas": "Região Leiria - Caldas"
}

# Aplicar as correções aos dados
raw["Concelho origem do incidente"] = raw["Concelho origem do incidente"].str.strip().replace(correcoes)
raw

,Código do Relatório,Concelho origem do incidente,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017
7561,EDPD_2017_NOV_T_63,Santa Maria da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017
7562,EDPD_2017_NOV_T_64,Santa Maria da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017
7563,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017


- - -

In [20]:
raw.to_csv("EventosLimpos.csv", index=False)